In [1]:
import numpy as np
from copy import deepcopy
import dionysus as d
import scipy

In [ ]:
def SchurComp(M, ind):
    ind = [i if i >=0 else M.shape[0] + i for i in ind]
    nind = [i for i in range(M.shape[0]) if i not in ind]
    A = M[nind, :][:, nind]
    B = M[nind, :][:, ind]
    C = M[ind, :][:, nind]
    D = M[ind, :][:, ind]
    return A - B @ np.linalg.pinv(D) @ C

def combinatorial_Laplacian(Bqplus1, Bq):
    upLaplacian = Bqplus1@Bqplus1.T
    if Bq == 0:
        return upLaplacian
    else:
        downLaplacian = Bq.T@Bq
        return upLaplacian + downLaplacian

def persistent_Laplacian(Bqplus1: np.array, Bq: np.array, verb = False) -> np.array:
    """
    Inclusion of K in L. Assume the boundaries are ordered the same and first the boundaries in K appear in L.

    Bqplus1: (q+1)-boundary matrix of L.
    Bq: q-boundary matrix of K. (nqK if q=0)
    """
    if type(Bq) == int:
        nqK = Bq
        downL = 0
    else:
        nqK = Bq.shape[1]
        downL = Bq.T@Bq

    if type(Bqplus1) != int:
        upLaplacian = Bqplus1@Bqplus1.T
        nqL = Bqplus1.shape[0]
        IKL = list(range(nqK, nqL))
        upL = SchurComp(upLaplacian, IKL)
    else:
        upL = 0
    if verb:
        print("upL:\n", upL)
        eigval, eigvec = np.linalg.eig(upL)
        for i, val in enumerate(eigval):
             print(np.round(val,2), np.round(eigvec[:,i]/np.min(np.abs(eigvec[:,i][np.abs(eigvec[:,i]) > 1e-5])),2))
        print()
        print("downL:\n", downL)
        eigval, eigvec = np.linalg.eig(downL)
        for i, val in enumerate(eigval):
             print(np.round(val,2), np.round(eigvec[:,i]/np.min(np.abs(eigvec[:,i][np.abs(eigvec[:,i]) > 1e-5])),2))
        print()
    return upL + downL


In [41]:

f = d.Filtration()
# Drawn
simplices = [([0], 0), ([1], 1), ([2], 2), ([3], 3), ([1,0], 4), ([0,3], 5), ([1,3], 6), ([2,3], 7), ([0,2], 8), ([0,1,3], 9), ([0,2,3], 10)]

# Test
# simplices = [([0], 0), ([1], 0), ([2], 0), ([3], 0), ([0,2], 1), ([0,3], 1), ([2,3], 1), ([0,1], 2), ([0,2,3], 3)]
for vertices, time in simplices:
    f.append(d.Simplex(vertices, time))
f.sort()
max_time = int(f[len(f)-1].data)

In [169]:
def compute_boundary_matrices(f: d.Filtration, weight_fun):
    t_old = 0
    relevant_times = [t_old]
    n_simplicies_seen_per_time = [[0]]
    n_simplicies_seen_total = [0]

    for s in f:
        q = s.dimension()
        if s.data != t_old:
            relevant_times.append(s.data)
            n_simplicies_seen_per_time.append(deepcopy(n_simplicies_seen_total))
            t_old = s.data

        if q >= len(n_simplicies_seen_total):
            n_simplicies_seen_total.append(0)
        n_simplicies_seen_total[q] += 1

    relevant_times.append(s.data)
    n_simplicies_seen_per_time.append(deepcopy(n_simplicies_seen_total))
    maxq = len(n_simplicies_seen_per_time[-1])
    for qi in range(len(n_simplicies_seen_per_time)):
        n_simplicies_seen_per_time[qi] = n_simplicies_seen_per_time[qi] + [0]*(maxq-len(n_simplicies_seen_per_time[qi]))

    def simplices_at_time(t):
        i = 0
        if t == np.inf:
            return n_simplicies_seen_per_time[-1]
        while relevant_times[i] <= t:
            i += 1
            if i == len(relevant_times) - 1:
                return n_simplicies_seen_per_time[-1]
        return n_simplicies_seen_per_time[i]

    simplices_at_end = simplices_at_time(np.inf)
    boundary_matrices = [0]+[np.zeros((simplices_at_end[q-1], simplices_at_end[q])) for q in range(1,maxq)]
    name_to_idx = [{} for _ in range(maxq)]
    for s in f:
        q = s.dimension()

        name = str(s).split("<")[1].split(">")[0]
        idx = len(name_to_idx[q])
        name_to_idx[q][name] = idx

        if q > 0:
            for i, bdry_simplex in enumerate(s.boundary()):
                name_bdry_simplex = str(bdry_simplex).split("<")[1].split(">")[0]  
                idx_bdry_simplex = name_to_idx[q-1][name_bdry_simplex]

                # Boundaries as described in OG paper
                boundary_matrices[q][idx_bdry_simplex, idx] = weight_fun(name)/weight_fun(name_bdry_simplex)*(-1)**i

                # Assuming product weights
                # for v in s:
                #     if v not in bdry_simplex:
                #         name_bdry_simplex = str(v)
                #         break
                # boundary_matrices[q][idx_bdry_simplex, idx] = weight_fun(name_bdry_simplex)*(-1)**i

    return boundary_matrices, name_to_idx, simplices_at_time

def weight_fun(x):
    # if x in ["0,2,3"]:
    if x in ["0,2,3"]:
        return 1
    return 1

boundary_matrices, name_to_idx, simplices_at_time = compute_boundary_matrices(f, weight_fun)
boundary_matrices

[0,
 array([[-1., -1.,  0.,  0., -1.],
        [ 1.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.],
        [ 0.,  1.,  1.,  1.,  0.]]),
 array([[ 1.,  0.],
        [-1., -1.],
        [ 1.,  0.],
        [ 0.,  1.],
        [ 0.,  1.]])]

In [170]:
def persistent_Laplacian_filtration(q, boundary_matrices, s, t, simplices_at_time, verb=False):
    if q > 0:
        Bq = boundary_matrices[q][:simplices_at_time(s)[q-1], :simplices_at_time(s)[q]]
    else:
        Bq = simplices_at_time(s)[q]
    
    if q+1 <= len(simplices_at_time(t)):
        Bqplus1 = boundary_matrices[q+1][:simplices_at_time(t)[q], :simplices_at_time(t)[q+1]]
    else:
        Bqplus1 = 0
    return persistent_Laplacian(Bqplus1, Bq, verb=verb)

for q in range(2):
    print("q:", q)
    for t in range(1,max_time+1):
        for s in range(t):

            Lap = persistent_Laplacian_filtration(q, boundary_matrices, s, t-1, simplices_at_time)
            evals = np.linalg.eig(Lap)[0]
            non_zero_evals = evals[evals > 1e-5]

            if len(non_zero_evals) > 0:
                bijm1_min = np.min(non_zero_evals)
            else:
                bijm1_min = 0

            bijm1 = np.sum(evals < 1e-5)

            Lap = persistent_Laplacian_filtration(q, boundary_matrices, s, t, simplices_at_time)
            evals = np.linalg.eig(Lap)[0]
            # if len(evals) > 0 :
            #     print(f"s: {s}, t: {t}, \tmin eval: {np.min(evals)}")
            non_zero_evals = evals[evals > 1e-5]
            
            if len(non_zero_evals) > 0:
                bij_min = np.min(non_zero_evals)
            else:
                bij_min = 0

            bij = np.sum(evals < 1e-5)

            Lap = persistent_Laplacian_filtration(q, boundary_matrices, s-1, t-1, simplices_at_time)
            evals = np.linalg.eig(Lap)[0]
            non_zero_evals = evals[evals > 1e-5]
                    
            if len(non_zero_evals) > 0:
                bim1jm1_min = np.min(non_zero_evals)
            else:
                bim1jm1_min = 0

            bim1jm1 = np.sum(evals < 1e-5)

            Lap = persistent_Laplacian_filtration(q, boundary_matrices, s-1, t, simplices_at_time)
            evals = np.linalg.eig(Lap)[0]
            non_zero_evals = evals[evals > 1e-5]
            
            if len(non_zero_evals) > 0:
                bim1j_min = np.min(non_zero_evals)
            else:
                bim1j_min = 0

            bim1j = np.sum(evals < 1e-5)

            

            if bijm1-bij-(bim1jm1-bim1j) > 0:
                print("Barcode:", s, t)
                print("Multiplicity:", bijm1-bij-(bim1jm1-bim1j))
                print("evals:", np.round(bijm1_min, 2), np.round(bij_min, 2), np.round(bim1jm1_min, 2), np.round(bim1j_min, 2))
                print("Same Formula:", np.round(bijm1_min-bij_min-(bim1jm1_min-bim1j_min), 2))
                print()
    print("-"*20)

q: 0
Barcode: 1 4
Multiplicity: 1
evals: 0 2.0 0 0
Same Formula: -2.0

Barcode: 3 5
Multiplicity: 1
evals: 2.0 1.0 2.0 2.0
Same Formula: 1.0

Barcode: 2 7
Multiplicity: 1
evals: 3.0 1.0 3.0 3.0
Same Formula: 2.0

--------------------
q: 1
Barcode: 6 9
Multiplicity: 1
evals: 3.0 3.0 1.0 1.0
Same Formula: -0.0

Barcode: 8 10
Multiplicity: 1
evals: 2.0 2.0 1.0 1.0
Same Formula: -0.0

--------------------


In [171]:
Lap = persistent_Laplacian_filtration(1, boundary_matrices, 9, 10, simplices_at_time, verb=True)
print(Lap)
eig_out = np.linalg.eig(Lap)
print("eigenvalues:", eig_out[0])
for i_zero in range(len(eig_out[0])):
    # if eig_out[0][i_zero] > 1e-5:
    #     continue
    print("i_chosen:", i_zero)
    eigval = eig_out[0][i_zero]
    eigvec = eig_out[1][:,i_zero]
    print(np.round(eigval,2), np.round(eigvec/np.min(np.abs(eigvec[np.abs(eigvec) > 1e-5])),2))
    print("mult:", eigvec*eigval)

upL:
 [[ 1. -1.  1.  0.  0.]
 [-1.  2. -1. -1. -1.]
 [ 1. -1.  1.  0.  0.]
 [ 0. -1.  0.  1.  1.]
 [ 0. -1.  0.  1.  1.]]
-0.0 [ 5.  2. -3.  1.  1.]
2.0 [ 1. -0.  1. -1. -1.]
4.0 [ 1. -2.  1.  1.  1.]
-0.0 [ 3.77 -2.   -5.77 -1.   -1.  ]
0.0 [ 0.  0.  0. -1.  1.]

downL:
 [[ 2.  1. -1.  0.  1.]
 [ 1.  2.  1.  1.  1.]
 [-1.  1.  2.  1.  0.]
 [ 0.  1.  1.  2. -1.]
 [ 1.  1.  0. -1.  2.]]
0.0 [ 3. -2.  3. -1. -1.]
4.0 [-3. -2.  1.  1. -3.]
2.0 [ 1. -0. -1.  1. -1.]
4.0 [  1.   -10.08 -11.08 -11.08   1.  ]
-0.0 [-4.5  1.  -4.5  3.5  3.5]

[[3. 0. 0. 0. 1.]
 [0. 4. 0. 0. 0.]
 [0. 0. 3. 1. 0.]
 [0. 0. 1. 3. 0.]
 [1. 0. 0. 0. 3.]]
eigenvalues: [4. 2. 4. 2. 4.]
i_chosen: 0
4.0 [1. 0. 0. 0. 1.]
mult: [2.82842712 0.         0.         0.         2.82842712]
i_chosen: 1
2.0 [-1.  0.  0.  0.  1.]
mult: [-1.41421356  0.          0.          0.          1.41421356]
i_chosen: 2
4.0 [0. 0. 1. 1. 0.]
mult: [0.         0.         2.82842712 2.82842712 0.        ]
i_chosen: 3
2.0 [ 0.  0. -1.  1.  0.]
mu

In [172]:
def persistent_Laplacian_new(Bqplus1: np.array, Bq: np.array, verb = False) -> np.array:
    """
    Inclusion of K in L. Assume the boundaries are ordered the same and first the boundaries in K appear in L.

    Bqplus1: (q+1)-boundary matrix of L.
    Bq: q-boundary matrix of K. (nqK if q=0)
    """
    if type(Bq) == int:
        nqK = Bq
        downL = 0
    else:
        nqK = Bq.shape[1]
        downL = Bq.T@Bq

    if type(Bqplus1) != int:
        upLaplacian = Bqplus1@Bqplus1.T
        nqL = Bqplus1.shape[0]
        IKL = list(range(nqK, nqL))
        upL = SchurComp(upLaplacian, IKL)
    else:
        upL = 0
    if verb:
        print("upL:\n", upL)
        if type(upL) != int:
            eigval, eigvec = np.linalg.eig(upL)
            for i, val in enumerate(eigval):
                print(np.round(val,2), np.round(eigvec[:,i]/np.min(np.abs(eigvec[:,i][np.abs(eigvec[:,i]) > 1e-5])),2))
        print()
        print("downL:\n", downL)
        if type(downL) != int:
            eigval, eigvec = np.linalg.eig(downL)
            for i, val in enumerate(eigval):
                print(np.round(val,2), np.round(eigvec[:,i]/np.min(np.abs(eigvec[:,i][np.abs(eigvec[:,i]) > 1e-5])),2))
        print()

    eigvals, eigvecs = np.linalg.eig(upL + downL)
    product = 1
    for i, eigval in enumerate(eigvals):
        product *= eigval
    
    eigvals, eigvecs = np.linalg.eig(upL + downL)
    betti = np.sum(eigvals<1e-5)

    return betti, product

def persistent_Laplacian_filtration_new(q, boundary_matrices, s, t, simplices_at_time, verb=False):
    if q > 0:
        Bq = boundary_matrices[q][:simplices_at_time(s)[q-1], :simplices_at_time(s)[q]]
    else:
        Bq = simplices_at_time(s)[q]
    
    if q+1 <= len(simplices_at_time(t)):
        Bqplus1 = boundary_matrices[q+1][:simplices_at_time(t)[q], :simplices_at_time(t)[q+1]]
    else:
        Bqplus1 = 0
    return persistent_Laplacian_new(Bqplus1, Bq, verb=verb)

In [173]:
for q in range(2):
    print("q:", q)
    for t in range(1,max_time+1):
        for s in range(t):

            bijm1, pijm1 = persistent_Laplacian_filtration_new(q, boundary_matrices, s, t-1, simplices_at_time, verb=False)

            bij, pij = persistent_Laplacian_filtration_new(q, boundary_matrices, s, t, simplices_at_time)

            bim1jm1, pim1jm1 = persistent_Laplacian_filtration_new(q, boundary_matrices, s-1, t-1, simplices_at_time)

            bim1j, pim1j = persistent_Laplacian_filtration_new(q, boundary_matrices, s-1, t, simplices_at_time)
            

            if bijm1-bij-(bim1jm1-bim1j) > 0:
                print("Barcode:", s, t)
                print("Multiplicity:", bijm1-bij-(bim1jm1-bim1j))
                print("evals:", np.round(pijm1, 2), np.round(pij, 2), np.round(pim1jm1, 2), np.round(pim1j, 2))
                print("Formula:", (pijm1/pij)/(pim1jm1/pim1j))
                print("New formula:", (pij/pijm1)/(pim1j/pim1jm1))
                print()
    print("-"*20)

q: 0
Barcode: 1 4
Multiplicity: 1
evals: 0.0 0.0 0.0 0.0
Formula: nan
New formula: nan

Barcode: 3 5
Multiplicity: 1
evals: 0.0 -0.0 0.0 0.0
Formula: nan
New formula: nan

Barcode: 2 7
Multiplicity: 1
evals: -0.0 0.0 -0.0 -0.0
Formula: -0.0
New formula: -inf

--------------------
q: 1
Barcode: 6 9
Multiplicity: 1
evals: -0.0 27.0 3.0 3.0
Formula: -1.480297366166875e-16
New formula: -6755399441055745.0

Barcode: 8 10
Multiplicity: 1
evals: -0.0 256.0 36.0 36.0
Formula: -1.6653345369377336e-16
New formula: -6004799503160666.0

--------------------


/tmp/ipykernel_859/985432896.py:19: RuntimeWarning: invalid value encountered in scalar divide
  print("Formula:", (pijm1/pij)/(pim1jm1/pim1j))
/tmp/ipykernel_859/985432896.py:20: RuntimeWarning: invalid value encountered in scalar divide
  print("New formula:", (pij/pijm1)/(pim1j/pim1jm1))
/tmp/ipykernel_859/985432896.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  print("New formula:", (pij/pijm1)/(pim1j/pim1jm1))


In [163]:
for q in range(2):
    print("q:", q)
    for t in range(1,max_time+1):
        for s in range(t):

            bijm1, pijm1 = persistent_Laplacian_filtration_new(q, boundary_matrices, s, t-1, simplices_at_time, verb=False)

            bij, pij = persistent_Laplacian_filtration_new(q, boundary_matrices, s, t, simplices_at_time)

            bim1jm1, pim1jm1 = persistent_Laplacian_filtration_new(q, boundary_matrices, s-1, t-1, simplices_at_time)

            bim1j, pim1j = persistent_Laplacian_filtration_new(q, boundary_matrices, s-1, t, simplices_at_time)
            

            if bijm1-bij-(bim1jm1-bim1j) > 0:
                print("Barcode:", s, t)
                print("betti:", bijm1,bij,bim1jm1,bim1j)
                print("evals:", np.round(pijm1, 2), np.round(pij, 2), np.round(pim1jm1, 2), np.round(pim1j, 2))
                print("Formula:", (pijm1/pij)/(pim1jm1/pim1j))
                print("New:", (pijm1+pij)/((pim1jm1+pim1j)/9))
    print("-"*20)

q: 0
Barcode: 1 4
betti: 2 1 1 1
evals: 0.0 0.0 0.0 0.0
Formula: nan
New: nan
Barcode: 3 5
betti: 3 2 2 2
evals: 0.0 -0.0 0.0 0.0
Formula: nan
New: nan
Barcode: 2 7
betti: 2 1 1 1
evals: -0.0 0.0 -0.0 -0.0
Formula: -0.0
New: -0.39193725585937467
--------------------
q: 1
Barcode: 6 9
betti: 1 0 0 0
evals: -0.0 27.0 3.0 3.0
Formula: -1.480297366166875e-16
New: 40.5
Barcode: 8 10
betti: 1 0 0 0
evals: -0.0 2304.0 36.0 36.0
Formula: -1.8503717077085935e-17
New: 288.0000000000001
--------------------


/tmp/ipykernel_859/1998660626.py:19: RuntimeWarning: invalid value encountered in scalar divide
  print("Formula:", (pijm1/pij)/(pim1jm1/pim1j))
/tmp/ipykernel_859/1998660626.py:20: RuntimeWarning: invalid value encountered in scalar divide
  print("New:", (pijm1+pij)/((pim1jm1+pim1j)/9))


In [39]:
Lap = persistent_Laplacian_filtration(0, boundary_matrices, 6, 7, simplices_at_time)
eig_out = np.linalg.eig(Lap)
print("eigenvalues:", eig_out[0])
for i_zero in range(len(eig_out[0])):
    if eig_out[0][i_zero] > 1e-5:
        continue
    print("i_chosen:", i_zero)
    eigval = eig_out[0][i_zero]
    eigvec = eig_out[1][:,i_zero]
    print(eigval, eigvec)


eigenvalues: [0. 0. 0. 1.]
i_chosen: 0
0.0 [1. 0. 0. 0.]
i_chosen: 1
0.0 [0. 1. 0. 0.]
i_chosen: 2
0.0 [0. 0. 1. 0.]


In [16]:
eigvec

array([-0.5547002, -0.5547002, -0.2773501, -0.5547002])

In [ ]:
# Fails because of machine precision !!!!!!!
simplices_at_time(0)

In [139]:
A = np.arange(6).reshape((3,2))
print(A, A.shape)
B = np.arange(4).reshape((2,2))
print(B, B.shape)
(A@B)

[[0 1]
 [2 3]
 [4 5]] (3, 2)
[[0 1]
 [2 3]] (2, 2)


array([[ 2,  3],
       [ 6, 11],
       [10, 19]])